In [ ]:
import json
import os
import zipfile
import numpy as np
from collections import defaultdict, Counter
from scipy.ndimage import label, binary_dilation, binary_erosion
from sklearn.cluster import KMeans
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
from typing import List, Dict, Tuple, Optional, Any
import random
import traceback
import warnings
import itertools
warnings.filterwarnings('ignore')

ARC_COLORS = {
    0: '#000000',
    1: '#0074D9',
    2: '#FF4136',
    3: '#2ECC40',
    4: '#FFDC00',
    5: '#AAAAAA',
    6: '#F012BE',
    7: '#FF851B',
    8: '#7FDBFF',
    9: '#870C25'
}

class EnhancedARCSolutionGenerator:
    def __init__(self, enable_visuals=True, genetic_generations=10):
        self.enable_visuals = enable_visuals
        self.genetic_generations = genetic_generations
        self.pattern_success_rates = defaultdict(lambda: {'attempts': 0, 'successes': 0})
        self.solution_complexity_scores = {}
        self.pattern_explanations = {}
        self.population = []
        self.fitness_history = []
        
        # Extended pattern handlers list with new patterns
        self.pattern_handlers = [
            # Original patterns
            (self.handle_outline_only, "Extract object outlines", "geometric"),
            (self.handle_center_object, "Center objects in grid", "geometric"),
            (self.handle_diagonal_mirror, "Diagonal mirror transformation", "geometric"),
            (self.handle_color_palette_row, "Extract color palette", "color"),
            (self.handle_bounding_fill_by_color, "Fill bounding boxes by color", "geometric"),
            (self.handle_row_col_propagation, "Propagate colors along rows/columns", "propagation"),
            (self.handle_repeat_pattern, "Tile pattern across grid", "tiling"),
            (self.handle_checkerboard, "Create checkerboard pattern", "tiling"),
            (self.handle_vertical_stripes, "Create vertical stripes", "tiling"),
            (self.handle_horizontal_stripes, "Create horizontal stripes", "tiling"),
            (self.handle_majority_color_fill, "Fill with majority color", "color"),
            (self.handle_cross_lines, "Draw cross lines", "geometric"),
            (self.handle_color_swap, "Swap two colors", "color"),
            (self.handle_crop_center, "Crop to center region", "geometric"),
            (self.handle_single_color_output, "Output single color", "color"),
            (self.handle_bounding_crop, "Crop to bounding box", "geometric"),
            (self.handle_overlay_fill, "Overlay fill pattern", "composite"),
            (self.handle_remove_color, "Remove specific color", "color"),
            (self.handle_color_mapping, "Map colors to new values", "color"),
            (self.handle_grid_operations, "Grid rotation/flip", "geometric"),
            (self.handle_resize, "Resize grid", "geometric"),
            (self.handle_object_operations, "Object translations", "geometric"),
            (self.handle_pattern_replication, "Replicate patterns", "tiling"),
            (self.handle_mirror_symmetry, "Mirror symmetry", "geometric"),
            (self.handle_arithmetic_operations, "Arithmetic operations", "arithmetic"),
            (self.handle_conditional_operations, "Conditional operations", "logic"),
            (self.handle_upscale_nearest, "Nearest neighbor upscaling", "scaling"),
            (self.handle_downscale_sample, "Downscale by sampling", "scaling"),
            (self.handle_fill_gaps_rowwise, "Fill gaps in rows", "propagation"),
            (self.handle_background_to_majority, "Change background to majority", "color"),
            (self.handle_keep_dominant_color, "Keep only dominant color", "color"),
            (self.handle_extract_first_nonempty_rowcol, "Extract first non-empty row/col", "extraction"),
            (self.handle_sort_rows_by_density, "Sort rows by density", "sorting"),
            (self.handle_sort_cols_by_density, "Sort columns by density", "sorting"),
            (self.handle_complete_by_mirror_half, "Complete by mirroring half", "geometric"),
            (self.handle_palette_column, "Extract color palette column", "color"),
            (self.handle_uniform_row_fill, "Fill rows uniformly", "propagation"),
            (self.handle_uniform_col_fill, "Fill columns uniformly", "propagation"),
            (self.handle_draw_frame, "Draw frame around grid", "geometric"),
            (self.handle_main_diag_line, "Draw main diagonal", "geometric"),
            (self.handle_anti_diag_line, "Draw anti-diagonal", "geometric"),
            (self.handle_border_to_zero, "Clear border pixels", "geometric"),
            (self.handle_single_object_translate, "Translate single object", "geometric"),
            (self.handle_object_count_row, "Count objects in row", "counting"),
            (self.handle_color_shift_plus_one, "Shift colors by +1", "arithmetic"),
            (self.handle_keep_colors, "Keep only specific colors", "color"),
            (self.handle_remove_small_objects, "Remove small objects", "filtering"),
            (self.handle_duplicate_quadrant, "Duplicate quadrant", "tiling"),
            
            # New patterns added
            (self.handle_flood_fill, "Flood fill from corners", "propagation"),
            (self.handle_extract_largest_object, "Extract largest connected component", "filtering"),
            (self.handle_count_objects, "Count connected components", "counting"),
            (self.handle_gravity_all_directions, "Apply gravity in all directions", "physics"),
            (self.handle_snake_pattern, "Snake/zigzag pattern", "geometric"),
            (self.handle_spiral_pattern, "Spiral pattern", "geometric"),
            (self.handle_pixel_art_double, "Pixel art doubling", "scaling"),
            (self.handle_extract_shape_template, "Extract shape as template", "extraction"),
            (self.handle_apply_mask, "Apply mask pattern", "composite"),
            (self.handle_color_gradient, "Color gradient fill", "color"),
            (self.handle_binary_operations, "Binary AND/OR/XOR", "logic"),
            (self.handle_convolution_patterns, "Convolution patterns", "filtering"),
            (self.handle_symmetry_detection, "Detect and apply symmetry", "geometric"),
            (self.handle_pattern_continuation, "Continue pattern", "propagation"),
            (self.handle_object_duplication, "Duplicate objects", "geometric"),
            (self.handle_color_inversion, "Invert colors (9-x)", "arithmetic"),
            (self.handle_extract_corners, "Extract corner regions", "extraction"),
            (self.handle_combine_grids, "Combine multiple grids", "composite"),
            (self.handle_sliding_window, "Sliding window operations", "filtering"),
            (self.handle_morphological_ops, "Morphological operations", "filtering"),
        ]

    # New pattern handlers
    def handle_flood_fill(self, task_data, analysis):
        """Flood fill from corners or specific positions"""
        try:
            a, b = map(np.array, (task_data['train'][0]['input'], task_data['train'][0]['output']))
            if a.shape != b.shape:
                return None
            
            # Check if it's a flood fill from corner
            if a[0,0] == 0 and b[0,0] != 0:
                fill_color = int(b[0,0])
                # Simple flood fill from top-left
                return f"""def p(g):
 H,W=len(g),len(g[0])
 out=[r[:] for r in g]
 stack=[(0,0)]
 out[0][0]={fill_color}
 while stack:
  y,x=stack.pop()
  for dy,dx in [(0,1),(1,0),(0,-1),(-1,0)]:
   ny,nx=y+dy,x+dx
   if 0<=ny<H and 0<=nx<W and out[ny][nx]==0:
    out[ny][nx]={fill_color}
    stack.append((ny,nx))
 return out
"""
        except:
            pass
        return None

    def handle_extract_largest_object(self, task_data, analysis):
        """Extract the largest connected component"""
        try:
            a, b = map(np.array, (task_data['train'][0]['input'], task_data['train'][0]['output']))
            
            # Label connected components
            labeled, num = label(a > 0)
            if num == 0:
                return None
            
            # Find largest component
            sizes = [np.sum(labeled == i) for i in range(1, num + 1)]
            largest = np.argmax(sizes) + 1
            
            # Check if output is just the largest component
            largest_mask = (labeled == largest).astype(int)
            result = a * largest_mask
            
            if np.array_equal(result, b):
                return """def p(g):
 from scipy.ndimage import label
 import numpy as np
 a=np.array(g)
 labeled,num=label(a>0)
 if num==0: return g
 sizes=[np.sum(labeled==i) for i in range(1,num+1)]
 largest=np.argmax(sizes)+1
 mask=(labeled==largest).astype(int)
 return (a*mask).tolist()
"""
        except:
            pass
        return None

    def handle_count_objects(self, task_data, analysis):
        """Count number of connected components"""
        try:
            a, b = (task_data['train'][0]['input'], task_data['train'][0]['output'])
            
            # Count objects in input
            labeled, num = label(np.array(a) > 0)
            
            # Check various output formats
            if len(b) == 1 and len(b[0]) == 1 and b[0][0] == num:
                return """def p(g):
 from scipy.ndimage import label
 import numpy as np
 _,n=label(np.array(g)>0)
 return [[n]]
"""
            
            # Output as row of length num
            if len(b) == 1 and len(b[0]) == num and all(x == 1 for x in b[0]):
                return """def p(g):
 from scipy.ndimage import label
 import numpy as np
 _,n=label(np.array(g)>0)
 return [[1]*n]
"""
        except:
            pass
        return None

    def handle_gravity_all_directions(self, task_data, analysis):
        """Apply gravity in different directions based on pattern"""
        try:
            a, b = (task_data['train'][0]['input'], task_data['train'][0]['output'])
            
            # Try gravity in each direction
            # Down
            down = [[0]*len(a[0]) for _ in range(len(a)-sum(1 for r in a if any(r)))]+[r for r in a if any(r)]
            if down == b:
                return """def p(g):
 return [[0]*len(g[0])for _ in range(len(g)-sum(1 for r in g if any(r)))]+[r for r in g if any(r)]
"""
            
            # Up
            up = [r for r in a if any(r)]+[[0]*len(a[0]) for _ in range(len(a)-sum(1 for r in a if any(r)))]
            if up == b:
                return """def p(g):
 return [r for r in g if any(r)]+[[0]*len(g[0])for _ in range(len(g)-sum(1 for r in g if any(r)))]
"""
            
            # Diagonal gravity (elements fall to bottom-left)
            h, w = len(a), len(a[0])
            result = [[0]*w for _ in range(h)]
            for i in range(h):
                for j in range(w):
                    if a[i][j] != 0:
                        # Find lowest position along diagonal
                        ni, nj = i, j
                        while ni + 1 < h and nj - 1 >= 0 and result[ni + 1][nj - 1] == 0:
                            ni += 1
                            nj -= 1
                        result[ni][nj] = a[i][j]
            if result == b:
                return """def p(g):
 h,w=len(g),len(g[0])
 r=[[0]*w for _ in range(h)]
 for i in range(h):
  for j in range(w):
   if g[i][j]!=0:
    ni,nj=i,j
    while ni+1<h and nj-1>=0 and r[ni+1][nj-1]==0:
     ni+=1;nj-=1
    r[ni][nj]=g[i][j]
 return r
"""
        except:
            pass
        return None

    def handle_snake_pattern(self, task_data, analysis):
        """Create snake/zigzag pattern"""
        try:
            out = task_data['train'][0]['output']
            h, w = len(out), len(out[0])
            
            # Check if it's a snake pattern with incrementing values
            expected = []
            for i in range(h):
                if i % 2 == 0:
                    expected.extend([i*w + j for j in range(w)])
                else:
                    expected.extend([i*w + w - 1 - j for j in range(w)])
            
            flat_out = [x for row in out for x in row]
            if all(flat_out[i] == expected[i] % 10 for i in range(len(flat_out))):
                return f"""def p(g):
 h,w={h},{w}
 r=[]
 for i in range(h):
  row=[]
  for j in range(w):
   if i%2==0: row.append((i*w+j)%10)
   else: row.append((i*w+w-1-j)%10)
  r.append(row)
 return r
"""
        except:
            pass
        return None

    def handle_spiral_pattern(self, task_data, analysis):
        """Create spiral pattern"""
        try:
            out = np.array(task_data['train'][0]['output'])
            h, w = out.shape
            
            # Generate spiral pattern and check if it matches
            spiral = [[0]*w for _ in range(h)]
            top, bottom, left, right = 0, h-1, 0, w-1
            num = 0
            
            while top <= bottom and left <= right:
                # Right
                for i in range(left, right + 1):
                    spiral[top][i] = num % 10
                    num += 1
                top += 1
                
                # Down
                for i in range(top, bottom + 1):
                    spiral[i][right] = num % 10
                    num += 1
                right -= 1
                
                # Left
                if top <= bottom:
                    for i in range(right, left - 1, -1):
                        spiral[bottom][i] = num % 10
                        num += 1
                    bottom -= 1
                
                # Up
                if left <= right:
                    for i in range(bottom, top - 1, -1):
                        spiral[i][left] = num % 10
                        num += 1
                    left += 1
            
            if np.array_equal(np.array(spiral), out):
                return f"""def p(g):
 h,w={h},{w}
 s=[[0]*w for _ in range(h)]
 t,b,l,r=0,h-1,0,w-1
 n=0
 while t<=b and l<=r:
  for i in range(l,r+1):s[t][i]=n%10;n+=1
  t+=1
  for i in range(t,b+1):s[i][r]=n%10;n+=1
  r-=1
  if t<=b:
   for i in range(r,l-1,-1):s[b][i]=n%10;n+=1
   b-=1
  if l<=r:
   for i in range(b,t-1,-1):s[i][l]=n%10;n+=1
   l+=1
 return s
"""
        except:
            pass
        return None

    def handle_pixel_art_double(self, task_data, analysis):
        """Double each pixel (different from regular 2x scaling)"""
        try:
            a, b = map(np.array, (task_data['train'][0]['input'], task_data['train'][0]['output']))
            
            # Check if each pixel becomes a 2x2 block with variations
            if b.shape[0] == a.shape[0] * 2 and b.shape[1] == a.shape[1] * 2:
                # Check for pattern where each pixel becomes a specific 2x2 pattern
                valid = True
                for i in range(a.shape[0]):
                    for j in range(a.shape[1]):
                        block = b[i*2:(i+1)*2, j*2:(j+1)*2]
                        # Check various patterns
                        if a[i,j] == 0:
                            if not np.all(block == 0):
                                valid = False
                                break
                        else:
                            # Cross pattern
                            if block[0,0] == a[i,j] and block[1,1] == a[i,j] and block[0,1] == 0 and block[1,0] == 0:
                                continue
                            # Inverse cross
                            elif block[0,1] == a[i,j] and block[1,0] == a[i,j] and block[0,0] == 0 and block[1,1] == 0:
                                continue
                            else:
                                valid = False
                                break
                    if not valid:
                        break
                
                if valid:
                    return """def p(g):
 h,w=len(g),len(g[0])
 r=[[0]*(w*2) for _ in range(h*2)]
 for i in range(h):
  for j in range(w):
   if g[i][j]!=0:
    r[i*2][j*2]=g[i][j]
    r[i*2+1][j*2+1]=g[i][j]
 return r
"""
        except:
            pass
        return None

    def handle_extract_shape_template(self, task_data, analysis):
        """Extract a specific shape pattern"""
        try:
            # Look for extraction of specific shapes (like L-shapes, T-shapes)
            a, b = map(np.array, (task_data['train'][0]['input'], task_data['train'][0]['output']))
            
            # If output is smaller, might be extracting a template
            if b.shape[0] < a.shape[0] or b.shape[1] < a.shape[1]:
                # Try to find b as a subregion in a
                for i in range(a.shape[0] - b.shape[0] + 1):
                    for j in range(a.shape[1] - b.shape[1] + 1):
                        if np.array_equal(a[i:i+b.shape[0], j:j+b.shape[1]], b):
                            return f"""def p(g):
 return [row[{j}:{j+b.shape[1]}] for row in g[{i}:{i+b.shape[0]}]]
"""
        except:
            pass
        return None

    def handle_apply_mask(self, task_data, analysis):
        """Apply a mask pattern to the input"""
        try:
            if len(task_data['train']) >= 2:
                # Assume first input is data, second input is mask
                a1 = np.array(task_data['train'][0]['input'])
                a2 = np.array(task_data['train'][1]['input']) if len(task_data['train']) > 1 else None
                b = np.array(task_data['train'][0]['output'])
                
                if a1.shape == b.shape:
                    # Check if output is input masked by non-zero positions
                    mask = (a1 > 0).astype(int)
                    if np.array_equal(b, a1 * mask):
                        return """def p(g):
 return [[x if x>0 else 0 for x in r] for r in g]
"""
        except:
            pass
        return None

    def handle_color_gradient(self, task_data, analysis):
        """Generate color gradients"""
        try:
            out = np.array(task_data['train'][0]['output'])
            h, w = out.shape
            
            # Radial gradient from center
            cy, cx = h // 2, w // 2
            radial = np.zeros((h, w))
            for i in range(h):
                for j in range(w):
                    dist = int(np.sqrt((i - cy)**2 + (j - cx)**2))
                    radial[i, j] = min(dist, 9)
            
            if np.array_equal(out, radial.astype(int)):
                return f"""def p(g):
 h,w={h},{w}
 cy,cx=h//2,w//2
 import numpy as np
 r=[]
 for i in range(h):
  row=[]
  for j in range(w):
   d=int(np.sqrt((i-cy)**2+(j-cx)**2))
   row.append(min(d,9))
  r.append(row)
 return r
"""
        except:
            pass
        return None

    def handle_binary_operations(self, task_data, analysis):
        """Binary operations between grids"""
        try:
            if len(task_data['train']) >= 2:
                # AND operation
                inp = task_data['train'][0]['input']
                out = task_data['train'][0]['output']
                
                # Self AND (keep non-zero)
                and_result = [[1 if x > 0 else 0 for x in row] for row in inp]
                if and_result == out:
                    return """def p(g):
 return [[1 if x>0 else 0 for x in r] for r in g]
"""
        except:
            pass
        return None

    def handle_convolution_patterns(self, task_data, analysis):
        """Apply convolution-like patterns"""
        try:
            a, b = map(np.array, (task_data['train'][0]['input'], task_data['train'][0]['output']))
            
            if a.shape == b.shape:
                # Edge detection kernel
                h, w = a.shape
                result = np.zeros_like(a)
                for i in range(1, h-1):
                    for j in range(1, w-1):
                        if a[i,j] > 0:
                            # Count neighbors
                            neighbors = sum(1 for di in [-1,0,1] for dj in [-1,0,1] 
                                          if (di,dj) != (0,0) and a[i+di,j+dj] > 0)
                            if neighbors < 3:
                                result[i,j] = a[i,j]
                
                if np.array_equal(result, b):
                    return """def p(g):
 h,w=len(g),len(g[0])
 r=[[0]*w for _ in range(h)]
 for i in range(1,h-1):
  for j in range(1,w-1):
   if g[i][j]>0:
    n=sum(1 for di in [-1,0,1] for dj in [-1,0,1] if (di,dj)!=(0,0) and g[i+di][j+dj]>0)
    if n<3: r[i][j]=g[i][j]
 return r
"""
        except:
            pass
        return None

    def handle_symmetry_detection(self, task_data, analysis):
        """Detect and complete symmetry"""
        try:
            a, b = map(np.array, (task_data['train'][0]['input'], task_data['train'][0]['output']))
            
            if a.shape == b.shape:
                # Check if completing vertical symmetry
                h, w = a.shape
                if w % 2 == 0:
                    left_half = a[:, :w//2]
                    right_half_flipped = a[:, w//2:][:, ::-1]
                    
                    # Complete right side from left
                    result = np.hstack([left_half, left_half[:, ::-1]])
                    if np.array_equal(result, b):
                        return """def p(g):
 w=len(g[0])
 return [r[:w//2]+r[:w//2][::-1] for r in g]
"""
        except:
            pass
        return None

    def handle_pattern_continuation(self, task_data, analysis):
        """Continue a pattern to fill the grid"""
        try:
            a, b = (task_data['train'][0]['input'], task_data['train'][0]['output'])
            
            # Detect repeating pattern in first row/column and continue it
            if len(a) > 0 and len(a[0]) > 0:
                # Check row pattern
                first_row = a[0]
                pattern_len = 0
                for plen in range(1, len(first_row)//2 + 1):
                    if first_row[:plen] * (len(first_row)//plen) == first_row[:len(first_row)//plen*plen]:
                        pattern_len = plen
                        break
                
                if pattern_len > 0:
                    pattern = first_row[:pattern_len]
                    # Generate full grid with this pattern
                    result = []
                    for i in range(len(b)):
                        row = []
                        for j in range(len(b[0])):
                            row.append(pattern[j % pattern_len])
                        result.append(row)
                    
                    if result == b:
                        pattern_str = str(pattern)
                        return f"""def p(g):
 pat={pattern_str}
 h,w=len(g),len(g[0])
 return [[pat[j%{pattern_len}] for j in range(w)] for i in range(h)]
"""
        except:
            pass
        return None

    def handle_object_duplication(self, task_data, analysis):
        """Duplicate objects in specific patterns"""
        try:
            a, b = map(np.array, (task_data['train'][0]['input'], task_data['train'][0]['output']))
            
            # Check if objects are duplicated with offset
            labeled_a, num_a = label(a > 0)
            labeled_b, num_b = label(b > 0)
            
            if num_b == num_a * 2:  # Objects doubled
                # Find offset pattern
                return None  # Complex implementation
        except:
            pass
        return None

    def handle_color_inversion(self, task_data, analysis):
        """Invert colors using formula 9-x"""
        try:
            a, b = map(np.array, (task_data['train'][0]['input'], task_data['train'][0]['output']))
            
            if a.shape == b.shape:
                # Check if it's color inversion
                if np.all(b == 9 - a):
                    return """def p(g):
 return [[9-x for x in r] for r in g]
"""
                
                # Check modular arithmetic
                for mod in [2, 3, 4, 5, 6, 7, 8, 9]:
                    if np.all(b == a % mod):
                        return f"""def p(g):
 return [[x%{mod} for x in r] for r in g]
"""
        except:
            pass
        return None

    def handle_extract_corners(self, task_data, analysis):
        """Extract corner regions"""
        try:
            a, b = (task_data['train'][0]['input'], task_data['train'][0]['output'])
            
            # Check if output is just corners
            if len(b) == 2 and len(b[0]) == 2:
                h, w = len(a), len(a[0])
                corners = [[a[0][0], a[0][w-1]], [a[h-1][0], a[h-1][w-1]]]
                if corners == b:
                    return """def p(g):
 h,w=len(g),len(g[0])
 return [[g[0][0],g[0][w-1]],[g[h-1][0],g[h-1][w-1]]]
"""
        except:
            pass
        return None

    def handle_combine_grids(self, task_data, analysis):
        """Combine multiple grids using various operations"""
        # This would need multiple inputs, skip for now
        return None

    def handle_sliding_window(self, task_data, analysis):
        """Apply sliding window operations"""
        try:
            a, b = map(np.array, (task_data['train'][0]['input'], task_data['train'][0]['output']))
            
            # Max pooling with 2x2 window
            if b.shape[0] == a.shape[0]//2 and b.shape[1] == a.shape[1]//2:
                valid = True
                for i in range(b.shape[0]):
                    for j in range(b.shape[1]):
                        window = a[i*2:(i+1)*2, j*2:(j+1)*2]
                        if b[i,j] != np.max(window):
                            valid = False
                            break
                    if not valid:
                        break
                
                if valid:
                    return """def p(g):
 h,w=len(g)//2,len(g[0])//2
 r=[]
 for i in range(h):
  row=[]
  for j in range(w):
   row.append(max(g[i*2][j*2],g[i*2][j*2+1],g[i*2+1][j*2],g[i*2+1][j*2+1]))
  r.append(row)
 return r
"""
        except:
            pass
        return None

    def handle_morphological_ops(self, task_data, analysis):
        """Morphological operations like erosion/dilation"""
        try:
            a, b = map(np.array, (task_data['train'][0]['input'], task_data['train'][0]['output']))
            
            if a.shape == b.shape:
                # Dilation
                dilated = binary_dilation(a > 0).astype(int)
                if np.array_equal(b > 0, dilated):
                    return """def p(g):
 from scipy.ndimage import binary_dilation
 import numpy as np
 return (binary_dilation(np.array(g)>0).astype(int)*1).tolist()
"""
                
                # Erosion
                eroded = binary_erosion(a > 0).astype(int)
                if np.array_equal(b > 0, eroded):
                    return """def p(g):
 from scipy.ndimage import binary_erosion
 import numpy as np
 return (binary_erosion(np.array(g)>0).astype(int)*1).tolist()
"""
        except:
            pass
        return None

    # Fixed checkerboard handler
    def handle_checkerboard(self, task_data, analysis):
        try:
            out = np.array(task_data['train'][0]['output'])
            unique_values = np.unique(out)
            
            # Check if we have at least 2 unique values
            if len(unique_values) < 2:
                return None
                
            if len(unique_values) == 2:
                c0, c1 = int(unique_values[0]), int(unique_values[1])
                # Check if it's a checkerboard pattern
                is_checker = True
                for i in range(out.shape[0]):
                    for j in range(out.shape[1]):
                        expected = c0 if (i + j) % 2 == 0 else c1
                        if out[i, j] != expected:
                            is_checker = False
                            break
                    if not is_checker:
                        break
                
                if is_checker:
                    return f"""def p(g):
 return[[{c0} if (i+j)%2==0 else {c1} for j in range(len(g[0]))] for i in range(len(g))]
"""
        except:
            pass
        return None

    # Fixed arithmetic operations handler
    def handle_detect_arithmetic(self, in_grid, out_grid):
        """Fixed arithmetic detection with better error handling"""
        if in_grid.shape != out_grid.shape:
            return None
            
        try:
            # Check addition
            diff = out_grid - in_grid
            if np.all(diff == diff[0,0]):
                return ('add', int(diff[0,0]))
            
            # Check multiplication
            non_zero_mask = in_grid > 0
            if np.any(non_zero_mask):
                with np.errstate(divide='ignore', invalid='ignore'):
                    safe_in = np.where(in_grid == 0, 1, in_grid)
                    ratio = out_grid / safe_in
                    # Only check ratio where input is non-zero
                    ratio_values = ratio[non_zero_mask]
                    if len(ratio_values) > 0 and np.all(ratio_values == ratio_values[0]):
                        return ('multiply', float(ratio_values[0]))
        except:
            pass
        return None

    # Original methods remain the same...
    def handle_uniform_row_fill(self, task_data, analysis):
        a,b = map(np.array,(task_data['train'][0]['input'],
                            task_data['train'][0]['output']))
        if a.shape!=b.shape: return None
        ok=True
        for r1,r2 in zip(a,b):
            maj = np.bincount(r1).argmax()
            if not np.all(r2==maj): ok=False; break
        if not ok: return None
        return """def p(g):
 return [[max(set(r), key=r.count)]*len(r) for r in g]
"""

    def handle_uniform_col_fill(self, task_data, analysis):
        a,b = map(np.array,(task_data['train'][0]['input'],
                            task_data['train'][0]['output']))
        if a.shape!=b.shape: return None
        H,W=a.shape
        ok=True
        for j in range(W):
            col=a[:,j].tolist()
            maj=max(set(col), key=col.count)
            if not np.all(b[:,j]==maj): ok=False; break
        if not ok: return None
        return """def p(g):
 H,W=len(g),len(g[0])
 cols=[[g[i][j] for i in range(H)] for j in range(W)]
 res=[[0]*W for _ in range(H)]
 for j,col in enumerate(cols):
  maj=max(set(col), key=col.count)
  for i in range(H): res[i][j]=maj
 return res
"""

    def handle_draw_frame(self, task_data, analysis):
        a,b = map(np.array,(task_data['train'][0]['input'],
                            task_data['train'][0]['output']))
        if b.shape[0]!=a.shape[0]+2 or b.shape[1]!=a.shape[1]+2: return None
        frame_color = int(b[0,0])
        inner = b[1:-1,1:-1]
        if np.array_equal(inner,a) and \
           np.all(b[[0,-1],:]==frame_color) and np.all(b[:,[0,-1]]==frame_color):
            return f"""def p(g):
 H,W=len(g),len(g[0])
 c={frame_color}
 out=[[c]*(W+2)]
 for r in g: out.append([c]+r+[c])
 out.append([c]*(W+2))
 return out
"""
        return None

    def handle_main_diag_line(self, task_data, analysis):
        a,b = map(np.array,(task_data['train'][0]['input'],
                            task_data['train'][0]['output']))
        if a.shape!=b.shape: return None
        H,W=a.shape
        if H!=W: return None
        diff = np.where(a!=b)
        if len(diff[0])==H and all(i==j for i,j in zip(*diff)):
            c=int(b[0,0])
            return f"""def p(g):
 n=len(g)
 return [[ {c} if i==j else g[i][j] for j in range(n)] for i in range(n)]
"""
        return None

    def handle_anti_diag_line(self, task_data, analysis):
        a,b = map(np.array,(task_data['train'][0]['input'],
                            task_data['train'][0]['output']))
        if a.shape!=b.shape: return None
        H,W=a.shape
        if H!=W: return None
        diff = np.where(a!=b)
        if len(diff[0])==H and all(i+j==H-1 for i,j in zip(*diff)):
            c=int(b[0,W-1])
            return f"""def p(g):
 n=len(g)
 return [[ {c} if i+j==n-1 else g[i][j] for j in range(n)] for i in range(n)]
"""
        return None

    def handle_border_to_zero(self, task_data, analysis):
        a,b = map(np.array,(task_data['train'][0]['input'],
                            task_data['train'][0]['output']))
        if a.shape!=b.shape: return None
        inner = a[1:-1,1:-1]
        if np.array_equal(b[1:-1,1:-1], inner) and np.all(b[[0,-1],:]==0) \
           and np.all(b[:,[0,-1]]==0):
            return """def p(g):
 H,W=len(g),len(g[0])
 out=[r[:] for r in g]
 for i in range(H):
  out[i][0]=out[i][-1]=0
 for j in range(W):
  out[0][j]=out[-1][j]=0
 return out
"""
        return None

    def handle_single_object_translate(self, task_data, analysis):
        try:
            if not analysis['object_properties'] or not analysis['object_properties'][0]:
                return None
            props = analysis['object_properties'][0][0]
            mv = props.get('position_change', None)
            if mv is None or isinstance(mv, str):
                return None
            dy, dx = map(int, mv)
            if (dy, dx) == (0, 0):
                return None
            return f'''def p(g):
 H, W = len(g), len(g[0])
 ys = [i for i,row in enumerate(g) for v in row if v]
 xs = [j for i,row in enumerate(g) for j,v in enumerate(row) if v]
 res = [[0]*W for _ in range(H)]
 for i,j in zip(ys, xs):
     ni, nj = i+{dy}, j+{dx}
     if 0 <= ni < H and 0 <= nj < W:
         res[ni][nj] = g[i][j]
 return res
'''
        except:
            return None

    def handle_object_count_row(self, task_data, analysis):
        a,b=(task_data['train'][0]['input'],task_data['train'][0]['output'])
        if len(b)!=1: return None
        flat=[x for r in a for x in r if x!=0]
        if not flat: return None
        maj=max(set(flat), key=flat.count)
        cnt=flat.count(maj)
        if b==[[maj]*cnt]:
            return f"""def p(g):
 flat=[x for r in g for x in r if x!=0]
 maj=max(set(flat), key=flat.count)
 return [[maj]*flat.count(maj)]
"""
        return None

    def handle_color_shift_plus_one(self, task_data, analysis):
        a,b=(task_data['train'][0]['input'],task_data['train'][0]['output'])
        if [[(x+1)%10 for x in r] for r in a]==b:
            return """def p(g):
 return [[(x+1)%10 for x in r] for r in g]
"""
        return None

    def handle_keep_colors(self, task_data, analysis):
        keep=set(x for ex in task_data['train']
                   for r in ex['output'] for x in r)
        if len(keep)>=10: return None
        a,b=(task_data['train'][0]['input'],task_data['train'][0]['output'])
        if [[x if x in keep else 0 for x in r] for r in a]==b:
            keep_str=",".join(map(str,keep))
            return f"""def p(g):
 keep={{ {keep_str} }}
 return [[x if x in keep else 0 for x in r] for r in g]
"""
        return None

    def handle_remove_small_objects(self, task_data, analysis):
        a,b=map(np.array,(task_data['train'][0]['input'],
                          task_data['train'][0]['output']))
        if a.shape!=b.shape: return None
        diff=a!=b
        if not diff.any(): return None
        ys,xs=np.where(diff)
        if len(ys)<=2 and np.all(b[ys,xs]==0):
            coords=[(int(y),int(x)) for y,x in zip(ys,xs)]
            return f"""def p(g):
 g=[r[:] for r in g]
 for y,x in {coords}: g[y][x]=0
 return g
"""
        return None

    def handle_duplicate_quadrant(self, task_data, analysis):
        a,b=map(np.array,(task_data['train'][0]['input'],
                          task_data['train'][0]['output']))
        h,w=a.shape
        if h%2 or w%2 or b.shape!=(h,w): return None
        q=a[:h//2,:w//2]
        cand=np.block([[q,q],[q,q]])
        if np.array_equal(cand,b):
            return """def p(g):
 h,w=len(g),len(g[0]);hh,ww=h//2,w//2
 q=[row[:ww] for row in g[:hh]]
 return [q_row+q_row for q_row in q]+[q_row+q_row for q_row in q]
"""
        return None

    def handle_upscale_nearest(self, task_data, analysis):
        a, b = map(np.array, (task_data['train'][0]['input'], task_data['train'][0]['output']))
        hi, wi = a.shape; ho, wo = b.shape
        if ho%hi or wo%wi: return None
        ky, kx = ho//hi, wo//wi
        ok = True
        for i in range(hi):
            for j in range(wi):
                block = b[i*ky:(i+1)*ky, j*kx:(j+1)*kx]
                if not np.all(block == a[i,j]): ok=False; break
            if not ok: break
        if not ok: return None
        return f"""def p(g):
 ky, kx = {ky}, {kx}
 H, W = len(g), len(g[0])
 return [[ g[i//ky][j//kx] for j in range(W*kx) ] for i in range(H*ky)]
"""

    def handle_downscale_sample(self, task_data, analysis):
        a, b = map(np.array, (task_data['train'][0]['input'], task_data['train'][0]['output']))
        hi, wi = a.shape; ho, wo = b.shape
        if hi%ho or wi%wo: return None
        ky, kx = hi//ho, wi//wo
        if not np.all(b == a[::ky, ::kx]): return None
        return f"""def p(g):
 ky, kx = {ky}, {kx}
 return [[ g[i*ky][j*kx] for j in range(len(g[0])//kx) ] for i in range(len(g)//ky)]
"""

    def handle_fill_gaps_rowwise(self, task_data, analysis):
        a, b = map(np.array, (task_data['train'][0]['input'], task_data['train'][0]['output']))
        if a.shape != b.shape: return None
        def fill_row(r):
            r = r[:]
            for c in set(x for x in r if x!=0):
                idx = [j for j,x in enumerate(r) if x==c]
                if idx:
                    l, rgt = min(idx), max(idx)
                    for j in range(l, rgt+1):
                        if r[j]==0: r[j]=c
            return r
        cand = np.array([fill_row(list(row)) for row in a.tolist()])
        if not np.array_equal(cand, b): return None
        return """def p(g):
 res=[]
 for r in g:
  r=r[:]
  s={x for x in r if x!=0}
  for c in s:
   idx=[j for j,x in enumerate(r) if x==c]
   l,rgt=min(idx),max(idx)
   for j in range(l,rgt+1):
    if r[j]==0: r[j]=c
  res.append(r)
 return res
"""

    def handle_background_to_majority(self, task_data, analysis):
        a, b = (task_data['train'][0]['input'], task_data['train'][0]['output'])
        def apply(g):
            flat=[x for r in g for x in r if x!=0]
            c=max(range(10), key=lambda v: flat.count(v)) if flat else 0
            return [[x if x!=0 else c for x in r] for r in g]
        if apply(a)==b:
            return """def p(g):
 flat=[x for r in g for x in r if x!=0]
 c=max(range(10), key=lambda v: flat.count(v)) if flat else 0
 return [[x if x!=0 else c for x in r] for r in g]
"""
        return None

    def handle_keep_dominant_color(self, task_data, analysis):
        a, b = (task_data['train'][0]['input'], task_data['train'][0]['output'])
        def apply(g):
            flat=[x for r in g for x in r if x!=0]
            if not flat: return [[0 for _ in r] for r in g]
            c=max(range(10), key=lambda v: flat.count(v))
            return [[x if x==c else 0 for x in r] for r in g]
        if apply(a)==b:
            return """def p(g):
 flat=[x for r in g for x in r if x!=0]
 if not flat: return [[0 for _ in r] for r in g]
 c=max(range(10), key=lambda v: flat.count(v))
 return [[x if x==c else 0 for x in r] for r in g]
"""
        return None

    def handle_extract_first_nonempty_rowcol(self, task_data, analysis):
        a, b = (task_data['train'][0]['input'], task_data['train'][0]['output'])
        for i, r in enumerate(a):
            if any(x!=0 for x in r):
                if b == [r]:
                    return """def p(g):
 for r in g:
  if any(x!=0 for x in r): return [r]
 return [g[0]]
"""
                break
        H, W = len(a), len(a[0])
        col = None
        for j in range(W):
            c = [a[i][j] for i in range(H)]
            if any(x!=0 for x in c):
                col = [[x] for x in c]; break
        if col and b == col:
            return """def p(g):
 H,W=len(g),len(g[0])
 for j in range(W):
  c=[g[i][j] for i in range(H)]
  if any(x!=0 for x in c): return [[x] for x in c]
 return [[g[i][0]] for i in range(H)]
"""
        return None

    def handle_sort_rows_by_density(self, task_data, analysis):
        a, b = (task_data['train'][0]['input'], task_data['train'][0]['output'])
        if len(a)!=len(b) or len(a[0])!=len(b[0]): return None
        asc  = sorted(a, key=lambda r: sum(x!=0 for x in r))
        desc = asc[::-1]
        if b==asc:
            return """def p(g):
 return sorted(g, key=lambda r: sum(x!=0 for x in r))
"""
        if b==desc:
            return """def p(g):
 return sorted(g, key=lambda r: sum(x!=0 for x in r), reverse=True)
"""
        return None

    def handle_sort_cols_by_density(self, task_data, analysis):
        a, b = map(np.array, (task_data['train'][0]['input'], task_data['train'][0]['output']))
        if a.shape != b.shape: return None
        H,W=a.shape
        def sort_cols(arr, rev=False):
            cols=[arr[:,j].tolist() for j in range(W)]
            cols=sorted(cols, key=lambda c: sum(x!=0 for x in c), reverse=rev)
            return [[cols[j][i] for j in range(W)] for i in range(H)]
        if np.array_equal(b, np.array(sort_cols(a, False))):
            return """def p(g):
 H,W=len(g),len(g[0])
 cols=[[g[i][j] for i in range(H)] for j in range(W)]
 cols=sorted(cols, key=lambda c: sum(x!=0 for x in c))
 return [[cols[j][i] for j in range(W)] for i in range(H)]
"""
        if np.array_equal(b, np.array(sort_cols(a, True))):
            return """def p(g):
 H,W=len(g),len(g[0])
 cols=[[g[i][j] for i in range(H)] for j in range(W)]
 cols=sorted(cols, key=lambda c: sum(x!=0 for x in c), reverse=True)
 return [[cols[j][i] for j in range(W)] for i in range(H)]
"""
        return None

    def handle_complete_by_mirror_half(self, task_data, analysis):
        a, b = map(np.array, (task_data['train'][0]['input'], task_data['train'][0]['output']))
        if a.shape != b.shape: return None
        H,W=a.shape
        mid=W//2
        left=a[:, :mid]
        cand_lr=np.hstack([left, np.fliplr(left)])
        if np.array_equal(b, cand_lr):
            return """def p(g):
 H,W=len(g),len(g[0]);m=W//2
 left=[r[:m] for r in g]
 return [ left[i]+left[i][::-1] for i in range(H) ]
"""
        mid=H//2
        top=a[:mid, :]
        cand_ud=np.vstack([top, np.flipud(top)])
        if np.array_equal(b, cand_ud):
            return """def p(g):
 H,W=len(g),len(g[0]);m=H//2
 top=g[:m]
 return top + top[::-1]
"""
        return None

    def handle_palette_column(self, task_data, analysis):
        inp, out = (task_data['train'][0]['input'], task_data['train'][0]['output'])
        colors = sorted({x for r in inp for x in r})
        if out == [[c] for c in colors]:
            return """def p(g):
 c=sorted({x for r in g for x in r})
 return [[x] for x in c]
"""
        return None
        
    def handle_repeat_pattern(self, task_data, analysis):
        inp = task_data['train'][0]['input']
        out = task_data['train'][0]['output']
        
        if not inp or not out or not inp[0] or not out[0]:
            return None
            
        if len(out) % len(inp) == 0 and len(out[0]) % len(inp[0]) == 0:
            ry = len(out) // len(inp)
            rx = len(out[0]) // len(inp[0])
            tiled = [[inp[i % len(inp)][j % len(inp[0])]
                      for j in range(len(inp[0]) * rx)]
                     for i in range(len(inp) * ry)]
            if tiled == out:
                return f"""def p(g):
 ry, rx = {ry}, {rx}
 h, w = len(g), len(g[0])
 return [[g[i % h][j % w] for j in range(w * rx)]
         for i in range(h * ry)]
"""
        return None

    def handle_vertical_stripes(self, task_data, analysis):
        inp,out=(np.array(task_data['train'][0][k]) for k in('input','output'))
        if out.shape!=inp.shape: return None
        for j in range(out.shape[1]):
            if len(set(out[:,j]))==1 and not np.array_equal(inp[:,j],out[:,j]):
                return f"""def p(g):
 H,W=len(g),len(g[0])
 return[[g[i][0] for _ in range(W)] for i in range(H)]
"""
        return None

    def handle_horizontal_stripes(self, task_data, analysis):
        inp,out=(np.array(task_data['train'][0][k]) for k in('input','output'))
        if out.shape!=inp.shape: return None
        for i in range(out.shape[0]):
            if len(set(out[i]))==1 and not np.array_equal(inp[i],out[i]):
                return """def p(g):
 return[[x for _ in g[0]] for x in [r[0] for r in g]]
"""
        return None

    def handle_majority_color_fill(self, task_data, analysis):
        inp, out=(task_data['train'][0]['input'],task_data['train'][0]['output'])
        flat=[x for r in inp for x in r]
        maj=max(set(flat), key=flat.count)
        if all(all(x==maj for x in r) for r in out):
            return f"""def p(g):
 from collections import Counter
 flat=[x for r in g for x in r]
 maj=Counter(flat).most_common(1)[0][0]
 return[[maj]*len(g[0]) for _ in g]
"""
        return None

    def handle_cross_lines(self, task_data, analysis):
        inp,out=(np.array(task_data['train'][0][k]) for k in('input','output'))
        if out.shape!=inp.shape: return None
        H,W=out.shape; cy,cx=H//2,W//2
        if np.all(out[cy,:]==out[cy,0]) and np.all(out[:,cx]==out[0,cx]):
            c=int(out[cy,cx])
            return f"""def p(g):
 H,W=len(g),len(g[0]);cy,cx=H//2,W//2
 return[[{c} if i==cy or j==cx else g[i][j] for j in range(W)] for i in range(H)]
"""
        return None

    def handle_color_swap(self, task_data, analysis):
        inp,out=(task_data['train'][0]['input'],task_data['train'][0]['output'])
        in_set={x for r in inp for x in r}; out_set={x for r in out for x in r}
        if len(in_set)==len(out_set)==2 and in_set==out_set:
            a,b=sorted(in_set)
            return f"""def p(g):
 return[[{a} if x=={b} else ({b} if x=={a} else x) for x in r] for r in g]
"""
        return None

    def handle_crop_center(self, task_data, analysis):
        inp,out=(np.array(task_data['train'][0][k]) for k in ('input','output'))
        h,w=inp.shape; ho,wo=out.shape
        cy,cx=(h-ho)//2,(w-wo)//2
        if np.array_equal(out, inp[cy:cy+ho, cx:cx+wo]):
            return f"""def p(g):
 h,w=len(g),len(g[0]);ho,wo={ho},{wo}
 cy,cx=(h-ho)//2,(w-wo)//2
 return[g[i][cx:cx+wo] for i in range(cy,cy+ho)]
"""
        return None

    def handle_outline_only(self, task_data, analysis):
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        if inp.shape != out.shape: return None
        h, w = inp.shape
        edge = np.zeros_like(inp)
        for i in range(h):
            for j in range(w):
                if inp[i,j]==0: continue
                nbor = [(i+1,j),(i-1,j),(i,j+1),(i,j-1)]
                if any(not(0<=y<h and 0<=x<w) or inp[y,x]==0 for y,x in nbor):
                    edge[i,j] = inp[i,j]
        if np.array_equal(out, edge):
            return """def p(g):
 h,w=len(g),len(g[0])
 return[[ g[i][j] if g[i][j] and any( (y<0 or y>=h or x<0 or x>=w or g[y][x]==0)
        for y,x in ((i+1,j),(i-1,j),(i,j+1),(i,j-1)) )
        else 0 for j in range(w)] for i in range(h)]
"""
        return None

    def handle_center_object(self, task_data, analysis):
        inp = task_data['train'][0]['input']
        out = task_data['train'][0]['output']
        if len(inp) != len(out) or len(inp[0]) != len(out[0]):
            return None
        ys = [i for i, row in enumerate(inp) for v in row if v]
        xs = [j for row in inp for j, v in enumerate(row) if v]
        if not ys:
            return None
        y0, y1 = min(ys), max(ys) + 1
        x0, x1 = min(xs), max(xs) + 1
        crop = [r[x0:x1] for r in inp[y0:y1]]
        H, W = len(inp), len(inp[0])
        h, w = len(crop), len(crop[0])
        cy, cx = (H - h) // 2, (W - w) // 2
        canvas = [[0] * W for _ in range(H)]
        for i in range(h):
            canvas[cy + i][cx:cx + w] = crop[i][:]
        if canvas == out:
            return f'''def p(g):
 H, W = len(g), len(g[0])
 ys = [i for i,row in enumerate(g) for v in row if v]
 xs = [j for row in g for j,v in enumerate(row) if v]
 y0, y1 = min(ys), max(ys)+1
 x0, x1 = min(xs), max(xs)+1
 crop = [r[x0:x1] for r in g[y0:y1]]
 h, w = len(crop), len(crop[0])
 cy, cx = (H-h)//2, (W-w)//2
 res = [[0]*W for _ in range(H)]
 for i in range(h):
     res[cy+i][cx:cx+w] = crop[i][:]
 return res
'''

    def handle_diagonal_mirror(self, task_data, analysis):
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        if inp.shape[0]!=inp.shape[1]: return None
        if np.array_equal(out, inp.T):
            return """def p(g):
 return[list(r) for r in zip(*g)]
"""
        return None

    def handle_color_palette_row(self, task_data, analysis):
        inp, out = (task_data['train'][0]['input'], task_data['train'][0]['output'])
        colors = sorted({x for row in inp for x in row})
        if len(out)==1 and out[0]==colors:
            return """def p(g):
 c=sorted({x for r in g for x in r})
 return [c]
"""
        return None

    def handle_bounding_fill_by_color(self, task_data, analysis):
        inp = task_data['train'][0]['input']
        out = task_data['train'][0]['output']
        if len(inp) != len(out) or len(inp[0]) != len(out[0]):
            return None
        H, W = len(inp), len(inp[0])
        res = [[0] * W for _ in range(H)]
        colors = {v for row in inp for v in row if v}
        for c in colors:
            ys = [i for i, row in enumerate(inp) for v in row if v == c]
            xs = [j for i, row in enumerate(inp) for j, v in enumerate(row) if v == c]
            y0, y1 = min(ys), max(ys) + 1
            x0, x1 = min(xs), max(xs) + 1
            for i in range(y0, y1):
                for j in range(x0, x1):
                    res[i][j] = c
        if res == out:
            return '''def p(g):
 H, W = len(g), len(g[0])
 res = [[0]*W for _ in range(H)]
 colors = {v for row in g for v in row if v}
 for c in colors:
     ys = [i for i,row in enumerate(g) for v in row if v==c]
     xs = [j for i,row in enumerate(g) for j,v in enumerate(row) if v==c]
     y0,y1 = min(ys), max(ys)+1
     x0,x1 = min(xs), max(xs)+1
     for i in range(y0,y1):
         for j in range(x0,x1):
             res[i][j] = c
 return res
'''

    def handle_row_col_propagation(self, task_data, analysis):
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        if inp.shape!=out.shape: return None
        if all((np.unique(o).size==1) or np.array_equal(o, i)
               for i,o in zip(inp, out)):
            return """def p(g):
 return[[g[r][0] if len(set(g[r]))>1 else g[r][c]
         for c in range(len(g[0]))] for r in range(len(g))]
"""
        if all((np.unique(out[:,c]).size==1) or np.array_equal(out[:,c], inp[:,c])
               for c in range(inp.shape[1])):
            return """def p(g):
 H,W=len(g),len(g[0])
 cols=[[g[r][c] for r in range(H)] for c in range(W)]
 for c in range(W):
     if len(set(cols[c]))>1:
         cols[c]=[cols[c][0]]*H
 return[[cols[c][r] for c in range(W)] for r in range(H)]
"""
        return None

    def handle_single_color_output(self, task_data, analysis):
        cands = {int(x) for ex in task_data['train']
                       for row in ex['output'] for x in row}
        if len(cands) == 1:
            c = cands.pop()
            return f"""def p(g):
 return[[{c}]*len(g[0]) for _ in g]
"""
        return None

    def handle_bounding_crop(self, task_data, analysis):
        inp, out = map(np.array, (task_data['train'][0]['input'],
                                  task_data['train'][0]['output']))
        nz = np.argwhere(inp)
        if not len(nz):
            return None
        (y0,x0),(y1,x1) = nz.min(0), nz.max(0)+1
        if np.array_equal(out, inp[y0:y1, x0:x1]):
            return f"""def p(g):
 c=[(i,j)for i,r in enumerate(g)for j,x in enumerate(r)if x]
 y0=min(i for i,_ in c); y1=max(i for i,_ in c)+1
 x0=min(j for _,j in c); x1=max(j for _,j in c)+1
 return[g[i][x0:x1] for i in range(y0,y1)]
"""
        return None

    def handle_overlay_fill(self, task_data, analysis):
        inp, out = map(np.array, (task_data['train'][0]['input'],
                                  task_data['train'][0]['output']))
        if inp.shape != out.shape:
            return None
        if np.all(np.where(out==0, inp, out) == out):
            return """def p(g):
 return[[g[i][j] if r[j]==0 else r[j]
         for j in range(len(r))]
        for i,r in enumerate(g)]
"""
        return None

    def handle_remove_color(self, task_data, analysis):
        inp, out = map(np.array, (task_data['train'][0]['input'],
                                  task_data['train'][0]['output']))
        if inp.shape != out.shape:
            return None
        diff = inp != out
        removed = {int(inp[i,j]) for (i,j) in zip(*np.where(diff))}
        if len(removed)==1 and np.all(out[diff]==0):
            c = removed.pop()
            return f"""def p(g):
 return[[0 if x=={c} else x for x in r] for r in g]
"""
        return None
        
    def handle_color_mapping(self, task_data, analysis):
        color_map = {}
        for pos, changes in analysis['color_changes'].items():
            if len(changes) == 1:
                src, dest = next(iter(changes))
                if src not in color_map:
                    color_map[src] = dest
                elif color_map[src] != dest:
                    return None
        
        if not color_map:
            return None
            
        cases = "\n".join([f"    if x=={int(src)}: return {int(dest)}" for src, dest in color_map.items()])
        return f"""def p(g):
 return[[(lambda x:
{cases}
    else x)(x)for x in r]for r in g]
"""
    
    def handle_grid_operations(self, task_data, analysis):
        if not analysis['symmetry']:
            return None
        sym_type = analysis['symmetry']
        if sym_type == 'rotate_90':
            return """def p(g):
 return[list(r)for r in zip(*g[::-1])]
"""
        if sym_type == 'rotate_180':
            return """def p(g):
 return[r[::-1]for r in g[::-1]]
"""
        if sym_type == 'rotate_270':
            return """def p(g):
 return[list(r)for r in zip(*g)][::-1]
"""
        if sym_type == 'flip_vertical':
            return """def p(g):
 return g[::-1]
"""
        if sym_type == 'flip_horizontal':
            return """def p(g):
 return[r[::-1]for r in g]
"""
        return None

    def handle_resize(self, task_data, analysis):
        first_pair = task_data['train'][0]
        in_grid = np.array(first_pair['input'])
        out_grid = np.array(first_pair['output'])
        h_out, w_out = out_grid.shape
        h_in, w_in = in_grid.shape
        
        if (h_out < h_in) or (w_out < w_in):
            return f"""def p(g):
 return [row[:{w_out}] for row in g[:{h_out}]]
"""
        if (h_out > h_in) or (w_out > w_in):
            return f"""def p(g):
 return [row+[0]*({w_out}-len(row)) for row in g]+[[0]*{w_out} for _ in range({h_out}-len(g))]
"""
        return None

    def handle_object_operations(self, task_data, analysis):
        obj_changes = [obj for example in analysis['object_properties'] for obj in example]
        if not obj_changes:
            return None
        if len(obj_changes) == 1 and isinstance(obj_changes[0]['position_change'], np.ndarray):
            dy, dx = map(int, obj_changes[0]['position_change'])
            return f"""def p(g):
 h,w=len(g),len(g[0])
 return[[g[i-{dy}][j-{dx}] if 0<=i-{dy}<h and 0<=j-{dx}<w else 0 for j in range(w)]for i in range(h)]
"""
        return None
    
    def handle_pattern_replication(self, task_data, analysis):
        first_pair = task_data['train'][0]
        in_grid = np.array(first_pair['input'])
        out_grid = np.array(first_pair['output'])
        if (out_grid.shape[0] % in_grid.shape[0] == 0 and 
            out_grid.shape[1] % in_grid.shape[1] == 0):
            tiles_y = out_grid.shape[0] // in_grid.shape[0]
            tiles_x = out_grid.shape[1] // in_grid.shape[1]
            return f"""def p(g):
 return[[g[i%{in_grid.shape[0]}][j%{in_grid.shape[1]}]for j in range(len(g[0])*{tiles_x})]for i in range(len(g)*{tiles_y})]
"""
        return None
    
    def handle_mirror_symmetry(self, task_data, analysis):
        first_pair = task_data['train'][0]
        in_grid = np.array(first_pair['input'])
        out_grid = np.array(first_pair['output'])
        if np.array_equal(out_grid, in_grid[:, ::-1]) and in_grid.shape == out_grid.shape:
            return """def p(g):
 return[r[::-1]for r in g]
"""
        return None
    
    def handle_arithmetic_operations(self, task_data, analysis):
        if not analysis['arithmetic']:
            return None
        op, val = analysis['arithmetic']
        if op == 'add':
            return f"""def p(g):
 return[[x+{val}for x in r]for r in g]
"""
        if op == 'multiply':
            return f"""def p(g):
 return[[x*{val}for x in r]for r in g]
"""
        return None
    
    def handle_conditional_operations(self, task_data, analysis):
        first_pair = task_data['train'][0]
        in_grid = np.array(first_pair['input'])
        out_grid = np.array(first_pair['output'])
        if in_grid.shape != out_grid.shape:
            return None
        diff = out_grid - in_grid
        changed = np.where(diff != 0)
        for i,j in zip(*changed):
            if i > 0 and in_grid[i-1,j] == out_grid[i,j]:
                return """def p(g):
 return[[g[i-1][j]if i>0 and g[i][j]!=0 else g[i][j]for j in range(len(g[0]))]for i in range(len(g))]
"""
            if j > 0 and in_grid[i,j-1] == out_grid[i,j]:
                return """def p(g):
 return[[g[i][j-1]if j>0 and g[i][j]!=0 else g[i][j]for j in range(len(g[0]))]for i in range(len(g))]
"""
        return None

    def visualize_grid(self, grid: np.ndarray, title: str = "Grid", ax=None) -> None:
        if ax is None:
            fig, ax = plt.subplots(1, 1, figsize=(6, 6))
        
        color_matrix = np.zeros((*grid.shape, 3))
        for i in range(grid.shape[0]):
            for j in range(grid.shape[1]):
                color_hex = ARC_COLORS[int(grid[i, j]) % 10]
                color_matrix[i, j] = [int(color_hex[i:i+2], 16)/255 for i in (1, 3, 5)]
        
        ax.imshow(color_matrix, interpolation='nearest')
        ax.set_title(title)
        ax.grid(True, which='both', color='gray', linewidth=0.5)
        ax.set_xticks(np.arange(-0.5, grid.shape[1], 1), minor=True)
        ax.set_yticks(np.arange(-0.5, grid.shape[0], 1), minor=True)
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        
    def visualize_transformation(self, input_grid: np.ndarray, output_grid: np.ndarray, 
                               predicted_grid: Optional[np.ndarray] = None,
                               pattern_name: str = "") -> None:
        if not self.enable_visuals:
            return
            
        n_plots = 3 if predicted_grid is not None else 2
        fig, axes = plt.subplots(1, n_plots, figsize=(5*n_plots, 5))
        
        self.visualize_grid(input_grid, "Input", axes[0])
        self.visualize_grid(output_grid, "Expected Output", axes[1])
        
        if predicted_grid is not None:
            self.visualize_grid(predicted_grid, "Predicted Output", axes[2])
            # Handle shape mismatch
            if output_grid.shape == predicted_grid.shape:
                match_pct = np.mean(output_grid == predicted_grid) * 100
                axes[2].text(0.5, -0.1, f"Match: {match_pct:.1f}%", 
                            transform=axes[2].transAxes, ha='center')
            else:
                axes[2].text(0.5, -0.1, f"Shape mismatch: {output_grid.shape} vs {predicted_grid.shape}", 
                            transform=axes[2].transAxes, ha='center')
        
        if pattern_name:
            fig.suptitle(f"Pattern: {pattern_name}", fontsize=14)
        
        plt.tight_layout()
        plt.show()

    def analyze_pattern_distribution(self, task_data: Dict) -> Dict[str, Any]:
        pattern_counts = defaultdict(int)
        pattern_categories = defaultdict(int)
        
        for handler, name, category in self.pattern_handlers:
            try:
                analysis = self.analyze_task(task_data)
                solution = handler(task_data, analysis)
                if solution and self.verify_solution(solution, task_data):
                    pattern_counts[name] += 1
                    pattern_categories[category] += 1
            except:
                continue
        
        return {
            'pattern_counts': dict(pattern_counts),
            'category_counts': dict(pattern_categories),
            'total_patterns': sum(pattern_counts.values())
        }

    def visualize_pattern_analysis(self, analysis_results: List[Dict]) -> None:
        if not self.enable_visuals:
            return
            
        fig, axes = plt.subplots(2, 2, figsize=(15, 12))
        
        ax1 = axes[0, 0]
        patterns = list(self.pattern_success_rates.keys())[:20]
        success_rates = [self.pattern_success_rates[p]['successes'] / 
                        max(self.pattern_success_rates[p]['attempts'], 1) * 100 
                        for p in patterns]
        
        ax1.barh(patterns, success_rates)
        ax1.set_xlabel('Success Rate (%)')
        ax1.set_title('Pattern Success Rates')
        ax1.grid(True, alpha=0.3)
        
        ax2 = axes[0, 1]
        categories = defaultdict(int)
        for results in analysis_results:
            for cat, count in results.get('category_counts', {}).items():
                categories[cat] += count
                
        if categories:
            ax2.pie(categories.values(), labels=categories.keys(), autopct='%1.1f%%')
            ax2.set_title('Pattern Category Distribution')
        
        ax3 = axes[1, 0]
        if self.solution_complexity_scores:
            complexities = list(self.solution_complexity_scores.values())
            ax3.hist(complexities, bins=30, edgecolor='black')
            ax3.set_xlabel('Complexity Score')
            ax3.set_ylabel('Frequency')
            ax3.set_title('Solution Complexity Distribution')
            ax3.grid(True, alpha=0.3)
        
        ax4 = axes[1, 1]
        if self.fitness_history:
            generations = range(len(self.fitness_history))
            best_fitness = [gen['best'] for gen in self.fitness_history]
            avg_fitness = [gen['average'] for gen in self.fitness_history]
            
            ax4.plot(generations, best_fitness, 'b-', label='Best Fitness')
            ax4.plot(generations, avg_fitness, 'r--', label='Average Fitness')
            ax4.set_xlabel('Generation')
            ax4.set_ylabel('Fitness Score')
            ax4.set_title('Genetic Algorithm Progress')
            ax4.legend()
            ax4.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
        
    def calculate_solution_complexity(self, solution_code: str) -> float:
        lines = solution_code.strip().split('\n')
        char_count = len(solution_code)
        line_count = len(lines)
        
        loops = sum(1 for line in lines if 'for' in line or 'while' in line)
        conditionals = sum(1 for line in lines if 'if' in line)
        comprehensions = sum(1 for line in lines if '[' in line and 'for' in line)
        
        complexity = (char_count * 0.01 + 
                     line_count * 2 + 
                     loops * 5 + 
                     conditionals * 3 + 
                     comprehensions * 4)
        
        return complexity
        
    def genetic_optimize_solution(self, task_data: Dict, initial_solution: str) -> str:
        self.population = [initial_solution]
        
        for _ in range(19):
            variant = self.mutate_solution(initial_solution)
            self.population.append(variant)
        
        for generation in range(self.genetic_generations):
            fitness_scores = []
            for solution in self.population:
                fitness = self.evaluate_solution_fitness(solution, task_data)
                fitness_scores.append(fitness)
            
            self.fitness_history.append({
                'best': max(fitness_scores),
                'average': np.mean(fitness_scores),
                'worst': min(fitness_scores)
            })
            
            new_population = []
            
            elite_count = len(self.population) // 5
            elite_indices = np.argsort(fitness_scores)[-elite_count:]
            for idx in elite_indices:
                new_population.append(self.population[idx])
            
            while len(new_population) < len(self.population):
                parent1 = self.tournament_select(self.population, fitness_scores)
                parent2 = self.tournament_select(self.population, fitness_scores)
                
                if random.random() < 0.7:
                    child = self.crossover_solutions(parent1, parent2)
                else:
                    child = parent1
                    
                if random.random() < 0.3:
                    child = self.mutate_solution(child)
                    
                new_population.append(child)
            
            self.population = new_population
        
        final_fitness = [self.evaluate_solution_fitness(s, task_data) for s in self.population]
        best_idx = np.argmax(final_fitness)
        return self.population[best_idx]
        
    def evaluate_solution_fitness(self, solution: str, task_data: Dict) -> float:
        fitness = 0.0
        
        if self.verify_solution(solution, task_data):
            fitness += 100.0
        
        char_count = len(solution.replace(' ', '').replace('\n', ''))
        fitness += max(0, 50 - char_count * 0.1)
        
        complexity = self.calculate_solution_complexity(solution)
        fitness -= complexity * 0.01
        
        generalization_score = self.test_generalization(solution, task_data)
        fitness += generalization_score * 20
        
        return max(0, fitness)
        
    def test_generalization(self, solution: str, task_data: Dict) -> float:
        try:
            namespace = {}
            exec(solution, namespace)
            p = namespace['p']
            
            correct = 0
            total = 0
            
            for example in task_data['train']:
                input_grid = example['input']
                expected = example['output']
                actual = p(input_grid)
                if actual == expected:
                    correct += 1
                total += 1
                
                noisy_input = self.add_minor_noise(input_grid)
                try:
                    noisy_output = p(noisy_input)
                    if isinstance(noisy_output, list) and len(noisy_output) > 0:
                        correct += 0.5
                    total += 1
                except:
                    pass
                    
            return correct / max(total, 1)
        except:
            return 0.0
            
    def add_minor_noise(self, grid: List[List[int]]) -> List[List[int]]:
        noisy = [row[:] for row in grid]
        h, w = len(grid), len(grid[0]) if grid else 0
        
        changes = random.randint(0, min(2, h*w//10))
        for _ in range(changes):
            i = random.randint(0, h-1)
            j = random.randint(0, w-1)
            colors = {grid[y][x] for y in range(h) for x in range(w)}
            if len(colors) > 1:
                noisy[i][j] = random.choice(list(colors - {grid[i][j]}))
                
        return noisy
        
    def tournament_select(self, population: List[str], fitness_scores: List[float]) -> str:
        tournament_size = 3
        indices = random.sample(range(len(population)), tournament_size)
        tournament_fitness = [fitness_scores[i] for i in indices]
        winner_idx = indices[np.argmax(tournament_fitness)]
        return population[winner_idx]
        
    def crossover_solutions(self, parent1: str, parent2: str) -> str:
        lines1 = parent1.strip().split('\n')
        lines2 = parent2.strip().split('\n')
        
        if len(lines1) < 3 or len(lines2) < 3:
            return parent1
            
        child_lines = [lines1[0]]
        
        body1 = lines1[1:]
        body2 = lines2[1:]
        
        for i in range(max(len(body1), len(body2))):
            if random.random() < 0.5:
                if i < len(body1):
                    child_lines.append(body1[i])
            else:
                if i < len(body2):
                    child_lines.append(body2[i])
                    
        return '\n'.join(child_lines)
        
    def mutate_solution(self, solution: str) -> str:
        lines = solution.strip().split('\n')
        mutation_type = random.choice(['simplify', 'compress', 'reorder'])
        
        if mutation_type == 'simplify':
            for i, line in enumerate(lines):
                if 'for i in range(len(' in line:
                    lines[i] = line.replace('for i in range(len(', 'for i,_ in enumerate(')
                elif 'lambda' not in line and ':' in line and '=' in line:
                    parts = line.split('=', 1)
                    if len(parts) == 2 and ';' not in line:
                        lines[i] = parts[0] + '=' + parts[1].strip()
                        
        elif mutation_type == 'compress':
            for i, line in enumerate(lines):
                if i > 0:
                    lines[i] = line.replace(' ', '', 1)
                    
        elif mutation_type == 'reorder':
            if len(lines) > 3:
                idx1, idx2 = random.sample(range(1, len(lines)-1), 2)
                lines[idx1], lines[idx2] = lines[idx2], lines[idx1]
                
        return '\n'.join(lines)
        
    def explain_pattern(self, pattern_name: str, task_data: Dict, 
                       solution: str) -> Dict[str, Any]:
        explanation = {
            'pattern_name': pattern_name,
            'description': '',
            'key_features': [],
            'transformation_steps': [],
            'code_explanation': [],
            'visual_example': None
        }
        
        for handler, name, category in self.pattern_handlers:
            if name == pattern_name:
                explanation['description'] = f"{name} ({category} pattern)"
                explanation['category'] = category
                break
                
        example = task_data['train'][0]
        input_grid = np.array(example['input'])
        output_grid = np.array(example['output'])
        
        if input_grid.shape != output_grid.shape:
            explanation['key_features'].append(
                f"Shape change: {input_grid.shape} → {output_grid.shape}"
            )
            
        input_colors = set(input_grid.flatten())
        output_colors = set(output_grid.flatten())
        if input_colors != output_colors:
            explanation['key_features'].append(
                f"Color change: {input_colors} → {output_colors}"
            )
            
        lines = solution.strip().split('\n')
        for line in lines:
            if 'for' in line:
                explanation['code_explanation'].append("Uses iteration")
            if 'if' in line:
                explanation['code_explanation'].append("Contains conditional logic")
            if 'zip' in line:
                explanation['code_explanation'].append("Combines multiple sequences")
            if '[::-1]' in line:
                explanation['code_explanation'].append("Reverses sequences")
                
        return explanation
        
    def generate_detailed_report(self, task_id: str, task_data: Dict, 
                               solution: str, pattern_name: str) -> Dict[str, Any]:
        report = {
            'task_id': task_id,
            'pattern': pattern_name,
            'solution_length': len(solution),
            'complexity_score': self.calculate_solution_complexity(solution),
            'verification_status': 'PASSED' if self.verify_solution(solution, task_data) else 'FAILED',
            'generalization_score': self.test_generalization(solution, task_data),
            'explanation': self.explain_pattern(pattern_name, task_data, solution)
        }
        
        if pattern_name in self.pattern_success_rates:
            stats = self.pattern_success_rates[pattern_name]
            report['pattern_success_rate'] = stats['successes'] / max(stats['attempts'], 1)
            
        return report
    
    def analyze_task(self, task_data):
        """Improved task analysis with better error handling"""
        all_examples = task_data['train'] + task_data.get('test', []) + task_data.get('arc-gen', [])
        analysis = {
            'color_changes': defaultdict(set),
            'shape_changes': set(),
            'object_properties': [],
            'symmetry': None,
            'arithmetic': None,
            'pattern_metrics': {},
            'statistical_features': {}
        }
        
        for example in all_examples:
            in_grid = np.array(example['input'])
            out_grid = np.array(example['output'])
            
            analysis['shape_changes'].add((in_grid.shape, out_grid.shape))
            
            for (i,j), val in np.ndenumerate(in_grid):
                if i < out_grid.shape[0] and j < out_grid.shape[1]:
                    if in_grid[i,j] != out_grid[i,j]:
                        analysis['color_changes'][(i,j)].add((int(in_grid[i,j]), int(out_grid[i,j])))
            
            analysis['statistical_features']['input_colors'] = len(np.unique(in_grid))
            analysis['statistical_features']['output_colors'] = len(np.unique(out_grid))
            analysis['statistical_features']['input_density'] = np.mean(in_grid > 0)
            analysis['statistical_features']['output_density'] = np.mean(out_grid > 0)
            
            analysis['object_properties'].append(self.analyze_objects(in_grid, out_grid))
            
            if analysis['symmetry'] is None:
                analysis['symmetry'] = self.detect_symmetry(in_grid, out_grid)
            
            if analysis['arithmetic'] is None:
                analysis['arithmetic'] = self.detect_arithmetic(in_grid, out_grid)
        
        return analysis
    
    def analyze_objects(self, in_grid, out_grid):
        structure = np.ones((3,3), dtype=int)
        labeled_in, n_in = label(in_grid > 0, structure)
        labeled_out, n_out = label(out_grid > 0, structure)
        
        objects = []
        for i in range(1, n_in+1):
            in_obj = (labeled_in == i)
            out_obj = None
            if i <= n_out:
                out_obj = (labeled_out == i)
            objects.append({
                'in_size': int(in_obj.sum()),
                'out_size': int(out_obj.sum()) if out_obj is not None else 0,
                'position_change': self.detect_position_change(in_obj, out_obj)
            })
        return objects
    
    def detect_position_change(self, in_obj, out_obj):
        if out_obj is None:
            return "removed"
        in_pos = np.argwhere(in_obj).mean(axis=0)
        out_pos = np.argwhere(out_obj).mean(axis=0)
        return out_pos - in_pos
    
    def detect_symmetry(self, in_grid, out_grid):
        if np.array_equal(out_grid, np.rot90(in_grid, 1)):
            return 'rotate_90'
        if np.array_equal(out_grid, np.rot90(in_grid, 2)):
            return 'rotate_180'
        if np.array_equal(out_grid, np.rot90(in_grid, 3)):
            return 'rotate_270'
        if np.array_equal(out_grid, np.flipud(in_grid)):
            return 'flip_vertical'
        if np.array_equal(out_grid, np.fliplr(in_grid)):
            return 'flip_horizontal'
        return None
    
    def detect_arithmetic(self, in_grid, out_grid):
        """Fixed arithmetic detection"""
        if in_grid.shape != out_grid.shape:
            return None
            
        try:
            # Check addition
            diff = out_grid - in_grid
            if np.all(diff == diff[0,0]):
                return ('add', int(diff[0,0]))
            
            # Check multiplication
            non_zero_mask = in_grid > 0
            if np.any(non_zero_mask):
                with np.errstate(divide='ignore', invalid='ignore'):
                    safe_in = np.where(in_grid == 0, 1, in_grid)
                    ratio = out_grid / safe_in
                    # Only check ratio where input is non-zero
                    ratio_values = ratio[non_zero_mask]
                    if len(ratio_values) > 0 and np.all(ratio_values == ratio_values[0]):
                        return ('multiply', float(ratio_values[0]))
        except:
            pass
        return None
    
    def generate_solution(self, task_data):
        analysis = self.analyze_task(task_data)
        
        # Try each pattern handler
        for handler, name, category in self.pattern_handlers:
            self.pattern_success_rates[name]['attempts'] += 1
            
            try:
                solution = handler(task_data, analysis)
                if solution and self.verify_solution(solution, task_data):
                    self.pattern_success_rates[name]['successes'] += 1
                    
                    complexity = self.calculate_solution_complexity(solution)
                    self.solution_complexity_scores[name] = complexity
                    
                    # Optional: genetic optimization
                    if self.genetic_generations > 0:
                        solution = self.genetic_optimize_solution(task_data, solution)
                    
                    return solution, name
            except Exception as e:
                # Silent fail for pattern handlers
                continue
        
        # If no pattern works, try pattern combinations
        combined_solution = self.try_pattern_combinations(task_data, analysis)
        if combined_solution:
            return combined_solution, "Combined Pattern"
        
        # Fallback to identity
        return """def p(g):
 return [row[:] for row in g]
""", "Identity (Fallback)"
    
    def try_pattern_combinations(self, task_data, analysis):
        """Try combining multiple patterns"""
        # Example: Try rotation followed by crop
        try:
            inp = task_data['train'][0]['input']
            out = task_data['train'][0]['output']
            
            # Try all rotations
            for rot in [1, 2, 3]:
                rotated = np.rot90(np.array(inp), rot).tolist()
                # Check if rotated matches output shape
                if len(rotated) == len(out) and len(rotated[0]) == len(out[0]) and rotated == out:
                    if rot == 1:
                        return """def p(g):
 return [list(r) for r in zip(*g[::-1])]
"""
                    elif rot == 2:
                        return """def p(g):
 return [r[::-1] for r in g[::-1]]
"""
                    elif rot == 3:
                        return """def p(g):
 return [list(r) for r in zip(*g)][::-1]
"""
            
            # Try flip then scale
            h_flipped = [r[::-1] for r in inp]
            if len(out) == 2 * len(h_flipped) and len(out[0]) == 2 * len(h_flipped[0]):
                # Check if it's scaled h_flipped
                valid = True
                for i in range(len(h_flipped)):
                    for j in range(len(h_flipped[0])):
                        if out[i*2][j*2] != h_flipped[i][j] or out[i*2+1][j*2] != h_flipped[i][j]:
                            valid = False
                            break
                    if not valid:
                        break
                if valid:
                    return """def p(g):
 f=[r[::-1] for r in g]
 return [[f[i//2][j//2] for j in range(len(f[0])*2)] for i in range(len(f)*2)]
"""
        except:
            pass
        
        return None
    
    def verify_solution(self, solution_code, task_data):
        """Improved solution verification"""
        try:
            namespace = {}
            exec(solution_code, namespace)
            p = namespace['p']
            
            # Test on all available examples
            examples = task_data['train']
            if 'test' in task_data:
                examples.extend(task_data['test'])
            if 'arc-gen' in task_data:
                examples.extend(task_data['arc-gen'])
            
            # For efficiency, test a subset if too many examples
            if len(examples) > 5:
                test_indices = random.sample(range(len(examples)), min(5, len(examples)))
                test_examples = [examples[i] for i in test_indices]
            else:
                test_examples = examples
            
            for example in test_examples:
                input_grid = [row[:] for row in example['input']]  # Deep copy
                expected = example['output']
                
                try:
                    actual = p(input_grid)
                    if actual != expected:
                        return False
                except:
                    return False
                    
            return True
        except:
            return False

def create_enhanced_arc_solutions():
    generator = EnhancedARCSolutionGenerator(enable_visuals=True, genetic_generations=5)
    solutions = {}
    reports = []
    analysis_results = []
    
    print("🚀 Starting Enhanced ARC Solution Generation...\n")
    
    for task_num in range(1, 401):
        task_id = f"{task_num:03d}"
        try:
            with open(f"/kaggle/input/google-code-golf-2025/task{task_id}.json") as f:
                task_data = json.load(f)
            
            pattern_analysis = generator.analyze_pattern_distribution(task_data)
            analysis_results.append(pattern_analysis)
            
            solution, pattern_name = generator.generate_solution(task_data)
            solutions[task_id] = solution
            
            report = generator.generate_detailed_report(task_id, task_data, solution, pattern_name)
            reports.append(report)
            
            # Visualize first few tasks if enabled
            if task_num <= 5 and generator.enable_visuals:
                try:
                    example = task_data['train'][0]
                    input_grid = np.array(example['input'])
                    output_grid = np.array(example['output'])
                    
                    namespace = {}
                    exec(solution, namespace)
                    p = namespace['p']
                    predicted_grid = np.array(p(example['input']))
                    
                    generator.visualize_transformation(
                        input_grid, output_grid, predicted_grid, pattern_name
                    )
                except:
                    pass  # Skip visualization on error
            
            if report['verification_status'] == 'PASSED':
                print(f"Task {task_id} ✅ - Pattern: {pattern_name} - "
                      f"Complexity: {report['complexity_score']:.1f}")
            else:
                print(f"Task {task_id} ❌ - Used fallback")
                
        except Exception as e:
            print(f"Error processing task {task_id}: {str(e)}")
            traceback.print_exc()
            solutions[task_id] = """def p(g):
 return [row[:] for row in g]
"""
    
    print("\n📊 Generating analysis visualizations...")
    generator.visualize_pattern_analysis(analysis_results)
    
    # Create submission
    os.makedirs("/kaggle/working/submission", exist_ok=True)
    for task_id, code in solutions.items():
        with open(f"/kaggle/working/submission/task{task_id}.py", "w") as f:
            f.write(code)
    
    with zipfile.ZipFile("/kaggle/working/submission.zip", "w") as zipf:
        for task_id in solutions:
            zipf.write(f"/kaggle/working/submission/task{task_id}.py", 
                       f"task{task_id}.py")
    
    with open("/kaggle/working/detailed_reports.json", "w") as f:
        json.dump(reports, f, indent=2)
    
    print("\n📈 Summary Statistics:")
    print(f"Total tasks processed: {len(solutions)}")
    print(f"Successful patterns: {sum(1 for r in reports if r['verification_status'] == 'PASSED')}")
    print(f"Average complexity: {np.mean([r['complexity_score'] for r in reports]):.2f}")
    print(f"Average generalization: {np.mean([r['generalization_score'] for r in reports]):.2%}")
    
    print("\n🎯 Top 10 Most Successful Patterns:")
    pattern_stats = [(name, stats['successes'], stats['attempts']) 
                     for name, stats in generator.pattern_success_rates.items()
                     if stats['attempts'] > 0]
    pattern_stats.sort(key=lambda x: x[1], reverse=True)
    
    for i, (name, successes, attempts) in enumerate(pattern_stats[:10]):
        success_rate = successes / attempts * 100
        print(f"{i+1}. {name}: {successes}/{attempts} ({success_rate:.1f}%)")
    
    print("\n✨ Enhanced ARC solutions created: submission.zip")
    print("📄 Detailed reports saved: detailed_reports.json")

if __name__ == "__main__":
    create_enhanced_arc_solutions()